In [1]:
import torch
from transformers import ViTImageProcessor,ViTForImageClassification
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os


device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# Load the image processor and model
# Load the image processor and model
image_processor=ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
def vit_transform(image):
    return image_processor(images=image,return_tensors="pt")['pixel_values'].squeeze(0)

train_data_set=ImageFolder(root=r"C:\Users\Ahmed Pasha\Desktop\vit_cicd\data_set\train",transform=vit_transform)
valid_data_set=ImageFolder(root=r"C:\Users\Ahmed Pasha\Desktop\vit_cicd\data_set\val",transform=vit_transform)

train_loader=DataLoader(train_data_set,batch_size=12,shuffle=True)
val_laoder=DataLoader(valid_data_set,batch_size=12,shuffle=True)

c:\Users\Ahmed Pasha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
model=ViTForImageClassification.from_pretrained("google/vit-base-patch16-224",num_labels=6,ignore_mismatched_sizes=True)
model.to(device)

optimizer=torch.optim.AdamW(model.parameters(),lr=3e-5)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def train_model(model,loader,optimizer):
    model.train()
    train_loss=0

    for images,labels in loader:
        image=images.to(device)
        label=labels.to(device)

        optimizer.zero_grad()

        output=model(pixel_values=image,labels=label)
        loss=output.loss

        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
    return train_loss/len(loader)

def eval_model(model,loader):
    model.eval()
    total_loss=0
    correct=0
    total=0

    with torch.no_grad():
        for images,labels in loader:
            image=images.to(device)
            label=labels.to(device)

            output=model(pixel_values=image,labels=label)
            loss=output.loss
            logit=output.logits
            total_loss+=loss.item()
            predicted=torch.argmax(logit,dim=1)
            correct+=(predicted==label).sum().item()
            total+=label.size(0)
    accuracy=correct/total  
    return total_loss/len(loader),accuracy

            

In [ ]:
epochs = 5
best_accuracy = 0.0

os.makedirs("models", exist_ok=True)

for epoch in range(epochs):
    train_loss = train_model(model, train_loader, optimizer)
    val_loss, val_accuracy = eval_model(model,val_laoder)  # ✅ FIX

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # 🔥 SAVE BEST MODEL
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), "models/best_vit_model.pth")
        print("✅ Best model saved!")
s

Epoch 1/5
Train Loss: 0.2118
Validation Loss: 0.0175, Validation Accuracy: 0.9980
✅ Best model saved!
Epoch 2/5
Train Loss: 0.0053
Validation Loss: 0.0065, Validation Accuracy: 0.9988
✅ Best model saved!
Epoch 3/5
Train Loss: 0.0021
Validation Loss: 0.0042, Validation Accuracy: 0.9992
✅ Best model saved!
Epoch 4/5
Train Loss: 0.0012
Validation Loss: 0.0034, Validation Accuracy: 0.9992
Epoch 5/5
Train Loss: 0.0008
Validation Loss: 0.0030, Validation Accuracy: 0.9992
